In [1]:
import pandas as pd
import json
import requests
from requests.auth import HTTPBasicAuth
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

# Index Functions

In [2]:
def create_index(es, index_name, settings=None):
    """
    Create an Elasticsearch index
    @param es: an Elasticsearch object
    @param index_name: the name of the new index to be created
    @param settings: the index settings
    @return whether the index was created
    """
    is_created = False
    try:
        if es.indices.exists(index=index_name):
            es.indices.delete(index=index_name, ignore=[404])
        es.indices.create(index=index_name, body=settings)
        is_created = True
    except Exception as ex:
        print(str(ex))
    return is_created

In [3]:
class Elastic:
    """
    A convenience object to send HTTP requests to Elasticsearch
    """
    def __init__(self, endpoint, username, password):
        """
        @param endpoint: the URL of the Elasticsearch instance
        @param username: the Elasticsearch username 
        @param password: the Elasticsearch password
        """
        self.header = {'Content-Type': 'application/json', 'charset':'UTF-8'}
        #self.header={'Content-Type': '--data-binary application/x-ndjson'}
        self.endpoint = endpoint
        self.username = username
        self.password = password
        self.methods_mapping = {'get': requests.get, 
                                'put':requests.put, 
                                'post':requests.post, 
                                'delete':requests.delete}
        
    def curl(self, method, handle, json=None):
        """
        Sends an HTTP request to the Elasticsearch instanc
        @param method: can be 'get', 'put', 'post', 'delete'
        @param handle: the API handle to be appended to the Elasticsearch url
        @param json: the json payload of the HTTP request
        """
        http_method = self.methods_mapping[method.lower()]
        r = http_method(f'{self.endpoint}/{handle}', auth=HTTPBasicAuth(USER, PWD), 
                        headers=self.header, json=json)
        return r

# Connect to ES

In [4]:
USER = 'elastic'
PWD = 'mypassword'
index_name = 'tesco'
ES_ENDPOINT = 'http://localhost:9200'

# Preparing Data

In [5]:
df = pd.read_csv('../../data/tesco/tesco_inventory_clean.csv')
df = df.set_index('product_id')
docs = df.to_dict(orient='records') #to json format
doc_ids = df.index

# Creating an index
    mappings:
     - I explicitly set the type of the columns category, description and ingredients to be text, the type of the 
         remaining columns is handled dynamically.
    
    In the settings:
     - I set the number of shards that an index should have to 8. 
     - I set the number of replicas per primary shard to 3.
     - I set the analyzer to be a standard one using english stopwords.

In [19]:
# Index settings
settings_basic = {
    "mappings" : {
      "properties" : {
        "category" : {
          "type" : "text"
        },
        "description" : {
          "type" : "text"
        },
        "ingredients" : {
          "type" : "text"
        }
      }
  },
    "settings": {
        "number_of_shards": 8,
        "number_of_replicas": 3,
        "analysis": {
            "analyzer": {"std_english": {"type": "standard", "stopwords": "_english_" }}
        }
    }
}

    Below, I connect to my local ES node

In [12]:
#connect to the local elasticsearch node and authenticate
es = Elasticsearch([ES_ENDPOINT], http_auth=(USER, PWD))
#create an index
is_created = create_index(es, index_name, settings=settings_basic)
print(f'Index creation: {is_created}')

/home/jolu/miniconda3/envs/prod/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/tmp/ipykernel_30117/1221527122.py:13: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=index_name, body=settings)


Index creation: True


    I use the Elastic class to send HTTP requests to ES

In [14]:
e = Elastic(ES_ENDPOINT, USER, PWD)

In [15]:
# get the index details and settings
r = e.curl('get', index_name)
r.json()

{'tesco': {'aliases': {},
  'mappings': {'properties': {'category': {'type': 'text'},
    'description': {'type': 'text'},
    'ingredients': {'type': 'text'}}},
  'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '8',
    'provided_name': 'tesco',
    'creation_date': '1654221012191',
    'analysis': {'analyzer': {'std_english': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '3',
    'uuid': '4FIve4PjScGMWKyB-bkogA',
    'version': {'created': '7160299'}}}}}

# Bulk indexing
    For efficiency

In [20]:
# bulk indexing (via official API)

actions = [
  {
    "_index": index_name,
    "_id": doc_id,
    "_source": doc
  }
  for doc_id, doc in list(zip(doc_ids, docs))
]

# send actions in bulk (the API takes care of chunking them optimally)
bulk(es, actions)

(22596, [])

In [21]:
# get settings info of the selected index
es.indices.get_settings(index=index_name)

{'tesco': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '8',
    'provided_name': 'tesco',
    'creation_date': '1654221012191',
    'analysis': {'analyzer': {'std_english': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '3',
    'uuid': '4FIve4PjScGMWKyB-bkogA',
    'version': {'created': '7160299'}}}}}

    Point #1: I used multiple shards and bulk indexing for efficiency
    Point #2: Not taken into account. 
    Point #3: Handled in the settings basics. 
    Point #4: Handled in the mapper of the settings basics.
    Point #5: I didn't do this. 

# Queries

In [22]:
# exact match query
query = {
  "query": {
    "match_phrase": {
      "description": {
        "query": "Frijj banana milkshake"
      }
    }
  }
}
r = e.curl('get', f'tesco/_search', query)
r.json()

{'took': 26,
 'timed_out': False,
 '_shards': {'total': 8, 'successful': 8, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 21.523006,
  'hits': [{'_index': 'tesco',
    '_type': '_doc',
    '_id': '51665765',
    '_score': 21.523006,
    '_source': {'category': 'dairy',
     'description': 'Frijj Banana Milkshake 471Ml',
     'ingredients': '0.8% Fat <strong>Milk</strong> (94%) Sugar <strong>Buttermilk</strong> Powder Banana Flavouring Stabilisers (Carrageenan, Cellulose Gum) Colour (Carotenes)',
     'energy': 64.0,
     'fat': 1.0,
     'saturates': 0.6,
     'salt': 0.2,
     'sugars': 9.8,
     'protein': 0.0,
     'carbohydrate': 0.0,
     'fibre': 0.0,
     'avg_price': 1.3}}]}}

    Point #7: I define a match query above on the description field. 

In [23]:
# Full text query, multiple fields
query = {
"query": {
    "multi_match": {
            "query": "milk",
            "type": "most_fields",
            "fields": ["description^2", "ingredients"]
        }
    }
}
r = e.curl('get', f'tesco/_search', query)
r.json()

{'took': 37,
 'timed_out': False,
 '_shards': {'total': 8, 'successful': 8, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7076, 'relation': 'eq'},
  'max_score': 12.30955,
  'hits': [{'_index': 'tesco',
    '_type': '_doc',
    '_id': '70582338',
    '_score': 12.30955,
    '_source': {'category': 'sweets',
     'description': 'Aero Medium Milk Bubbly Milk Bar 40G',
     'ingredients': 'Sugar Dried Whole <strong>Milk</strong> Cocoa Mass Cocoa Butter Vegetable Fat (Palm/Shea/Illipe/Mango Kernel/Kokum Gurgi/Sal) Lactose and Proteins from Whey (from <strong>Milk</strong>) Whey Powder (from <strong>Milk</strong>) Emulsifier (Sunflower Lecithin) Flavouring Milk Chocolate contains Cocoa Solids 25% minimum Milk Solids 14% minimum and Vegetable Fat in addition to Cocoa Butter',
     'energy': 542.0,
     'fat': 30.9,
     'saturates': 18.0,
     'salt': 0.2,
     'sugars': 56.2,
     'protein': 6.5,
     'carbohydrate': 57.8,
     'fibre': 2.2,
     'avg_price': 0.6}},
   {'_index':

    Point #8: I define a multi-match query above on description and ingredients, boosting description by a factor of 2. 

In [24]:
# full-text query
query = {
  "query": {
    "bool": {
      "must": {
        "multi_match": {
          "query": "milk",
          "fields": [
            "description",
            "ingredients"
          ],
          "type": "best_fields"
        }
      },
      "filter": {
        "term": {
          "category": "sweets"
        }
      }
    }
  }
}
r = e.curl('get', f'tesco/_search', query)
r.json()

{'took': 16,
 'timed_out': False,
 '_shards': {'total': 8, 'successful': 8, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2675, 'relation': 'eq'},
  'max_score': 5.181798,
  'hits': [{'_index': 'tesco',
    '_type': '_doc',
    '_id': '70582338',
    '_score': 5.181798,
    '_source': {'category': 'sweets',
     'description': 'Aero Medium Milk Bubbly Milk Bar 40G',
     'ingredients': 'Sugar Dried Whole <strong>Milk</strong> Cocoa Mass Cocoa Butter Vegetable Fat (Palm/Shea/Illipe/Mango Kernel/Kokum Gurgi/Sal) Lactose and Proteins from Whey (from <strong>Milk</strong>) Whey Powder (from <strong>Milk</strong>) Emulsifier (Sunflower Lecithin) Flavouring Milk Chocolate contains Cocoa Solids 25% minimum Milk Solids 14% minimum and Vegetable Fat in addition to Cocoa Butter',
     'energy': 542.0,
     'fat': 30.9,
     'saturates': 18.0,
     'salt': 0.2,
     'sugars': 56.2,
     'protein': 6.5,
     'carbohydrate': 57.8,
     'fibre': 2.2,
     'avg_price': 0.6}},
   {'_index':

    Point #9: Same as 8 but I added a filter on category, in this case filtering by "sweets".

In [25]:
query ={
  "query": {
    "fuzzy": {
      "ingredients": {
        "value": "milk",
        "fuzziness": "AUTO",
        "max_expansions": 50,
        "prefix_length": 0,
        "transpositions": True,
        "rewrite": "constant_score"
      }
    }
  }
}
r = e.curl('get', f'tesco/_search', query)
r.json()

{'took': 30,
 'timed_out': False,
 '_shards': {'total': 8, 'successful': 8, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6906, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'tesco',
    '_type': '_doc',
    '_id': '62069816',
    '_score': 1.0,
    '_source': {'category': 'grains',
     'description': 'Tesco 6 Garlic And Coriander Mini Naan Bread 300G',
     'ingredients': 'Wheat Flour (<strong>Wheat</strong> Flour, Calcium Carbonate, Iron, Niacin, Thiamin) Water Rapeseed Oil Sugar Yeast Spirit Vinegar Kalonji Seeds Raising Agents (Disodium Diphosphate, Sodium Bicarbonate, Calcium Phosphate) Garlic (0.8%) Dried Parsley Salt Flavouring Preservative (Calcium Propionate) Dried Skimmed <strong>Milk</strong> Ground Coriander',
     'energy': 254.0,
     'fat': 3.5,
     'saturates': 0.3,
     'salt': 0.7,
     'sugars': 4.3,
     'protein': 7.4,
     'carbohydrate': 46.6,
     'fibre': 3.4,
     'avg_price': 1.275}},
   {'_index': 'tesco',
    '_type': '_doc',
   

In [26]:
# full-text query
query = {
    "query": {
        "match": {
            "description": {
                "query": "milk"
            }
        }
    },
   "sort" : [
      {"avg_price" : {"order" : "desc"}}
   ]
}
r = e.curl('get', f'tesco/_search', query)
r.json()

{'took': 38,
 'timed_out': False,
 '_shards': {'total': 8, 'successful': 8, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 504, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'tesco',
    '_type': '_doc',
    '_id': '76098401',
    '_score': None,
    '_source': {'category': 'dairy',
     'description': 'Alpro Longlife Unsweetened Soya Milk Alternative 6 X 1 Litre',
     'ingredients': 'Water Hulled <strong>Soya</strong> Beans (6.5%) Acidity Regulators (Monopotassium Phosphate, Dipotassium Phosphate) Calcium (Calcium Carbonate) Maltodextrin Stabiliser (Gellan Gum) Vitamins (Riboflavin (B2), B12, D2)',
     'energy': 32.0,
     'fat': 1.8,
     'saturates': 0.3,
     'salt': 0.03,
     'sugars': 0.1,
     'protein': 3.3,
     'carbohydrate': 0.2,
     'fibre': 0.6,
     'avg_price': 7.49},
    'sort': [7.49]},
   {'_index': 'tesco',
    '_type': '_doc',
    '_id': '59726704',
    '_score': None,
    '_source': {'category': 'sweets',
     'description': 'Lindt Li